In [16]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

# 1.라이브러리 및 데이터 로드

In [17]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:80% !important;}</style>"))
%matplotlib inline

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

mpl.rcParams['figure.figsize'] = (12,8)  #시각화 figure default 설정
mpl.rcParams['font.family'] = 'NanumGothic' #폰트 디폴트 설정
mpl.rcParams['font.size'] = 10    #폰트 사이즈 디폴트 설정
plt.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format='retina' # 그래프 글씨 뚜렷

In [18]:
import os
os.listdir()

['.config', '최종광진구0502.csv', 'catboost_info', 'sample_data']

In [19]:
df = pd.read_csv('최종광진구0502.csv')

In [20]:
round(df["전기차충전기유무"].value_counts() / df.shape[0] * 100, 2)

0    87.95
1    12.05
Name: 전기차충전기유무, dtype: float64

In [21]:
df

,fid,id,left,top,right,bottom,관광여가오락수,교육수,부동산수,생활서비스수,...,병원수,어린이집수,학교수_y,공영주차장수,전기차충전기수,급속충전기수,완속충전기수,전기차충전기유무,급속충전기유무,완속충전기유무
0,1,790,206153.585523,449218.165378,206253.585523,449118.165378,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
1,2,791,206153.585523,449118.165378,206253.585523,449018.165378,7,0,0,3,...,0,0,0,0,0,0,0,0,0,0
2,3,788,206153.585523,449418.165378,206253.585523,449318.165378,4,3,1,5,...,4,1,0,0,0,0,0,0,0,0
3,4,789,206153.585523,449318.165378,206253.585523,449218.165378,1,0,1,4,...,0,0,0,0,0,0,0,0,0,0
4,5,786,206153.585523,449618.165378,206253.585523,449518.165378,0,1,2,5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,1925,780,206153.585523,450218.165378,206253.585523,450118.165378,1,0,0,3,...,0,2,0,0,0,0,0,0,0,0
1091,1927,781,206153.585523,450118.165378,206253.585523,450018.165378,1,5,0,3,...,5,0,0,0,0,0,0,0,0,0
1092,1929,778,206153.585523,450418.165378,206253.585523,450318.165378,0,0,0,2,...,0,2,0,0,0,0,0,0,0,0
1093,1931,779,206153.585523,450318.165378,206253.585523,450218.165378,0,0,2,3,...,0,0,0,0,0,0,0,0,0,0


# 2.Hyperparameter Tuning

In [22]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
!pip install catboost
from catboost import CatBoostClassifier

import os
import random

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, accuracy_score, label_ranking_average_precision_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(100) # Seed 고정

In [24]:
# df.head(3)

In [25]:
X = df.loc[:,'관광여가오락수':'공영주차장수']
y = df['전기차충전기유무']

In [26]:
y.value_counts()

0    963
1    132
Name: 전기차충전기유무, dtype: int64

In [27]:
#  predictions = clf.predict(X_test)
# >>> cm = confusion_matrix(y_test, predictions, labels=clf.classes_)
# >>> disp = ConfusionMatrixDisplay(confusion_matrix=cm,
# ...                               display_labels=clf.classes_)

## 3.0 RandomClassifier

In [28]:
rfc = RandomForestClassifier(random_state = 100,
                             n_estimators = 1000,
                             class_weight= "balanced")

params = {"criterion" :  ["gini"],
          "min_samples_leaf" : [1]}

gs_rfc = GridSearchCV(estimator = rfc,
                      param_grid = params,
                      scoring = 'accuracy',
                      refit = True,
                      n_jobs = -1,
                      verbose = True)

gs_rfc.fit(X, y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(estimator=RandomForestClassifier(class_weight='balanced',
                                              n_estimators=1000,
                                              random_state=100),
             n_jobs=-1,
             param_grid={'criterion': ['gini'], 'min_samples_leaf': [1]},
             scoring='accuracy', verbose=True)

In [29]:
print(gs_rfc.best_score_)
print(gs_rfc.best_params_)
randomforest = gs_rfc.best_estimator_

0.8849315068493151
{'criterion': 'gini', 'min_samples_leaf': 1}


## 3.1XGBClassifier

In [30]:
xgb = XGBClassifier(random_state = 100,
                    n_estimators = 1000)

params = {"learning_rate" : [0.1, 0.3, 0.5, 0.7, 0.9],
          "max_depth" :  [10, 15, 20, 25, 30],
          "subsample" : [0.3, 0.5, 0.7, 0.9]}

gs_xgb = GridSearchCV(estimator = xgb,
                      param_grid = params,
                      scoring = 'accuracy',
                      refit = True,
                      n_jobs = -1,
                      verbose = True)

gs_xgb.fit(X, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: ignored

In [25]:
print(gs_xgb.best_score_)
print(gs_xgb.best_params_)
xgboost = gs_xgb.best_estimator_

0.8666666666666666
{'learning_rate': 0.1, 'max_depth': 10, 'subsample': 0.7}


## 3.2LGBMClassifier

In [26]:
lgb = LGBMClassifier(random_state = 100,
                    n_estimators = 1000)

params = {"learning_rate" : [0.1],
          "max_depth" :  [10, 15],
          "subsample" : [0.3]}

gs_lgb = GridSearchCV(estimator = lgb,
                      param_grid = params,
                      scoring = 'accuracy',
                      n_jobs = -1,
                      verbose = True)

gs_lgb.fit(X, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


GridSearchCV(estimator=LGBMClassifier(n_estimators=1000, random_state=100),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.3, 0.5, 0.7, 0.9],
                         'max_depth': [10, 15, 20, 25, 30],
                         'subsample': [0.3, 0.5, 0.7, 0.9]},
             scoring='accuracy', verbose=True)

In [27]:
print(gs_lgb.best_score_)
print(gs_lgb.best_params_)
lgbm = gs_lgb.best_estimator_

0.8666666666666666
{'learning_rate': 0.1, 'max_depth': 15, 'subsample': 0.3}


## 3.3CatboostClassifier


In [4]:
cb = CatBoostClassifier(random_state = 100, iterations = 10)

params = {"learning_rate" : [0.1, 0.3, 0.5, 0.7, 0.9],
          "max_depth" :  [10, 15, 20, 25, 30],
          "subsample" : [0.3, 0.5, 0.7, 0.9]}

gs_cb = RandomizedSearchCV(estimator = cb,
                            param_distributions= params,
                            n_iter = 10,
                            cv = 5,
                            scoring = 'accuracy',
                            n_jobs = -1,
                            verbose = True)

gs_cb.fit(X, y)

NameError: ignored

In [ ]:
print(gs_cb.best_score_)
print(gs_cb.best_params_)
catboost = gs_cb.best_estimator_

# 3.Oversampling with SMOTE

In [ ]:
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

# SMOTE 객체
sm = SMOTE(random_state = 100)

# oversampling
X_resampled, y_resampled = sm.fit_resample(X, y)

print('Before OverSampling, the shape of train_X: {}'.format(X.shape))
print('Before OverSampling, the shape of train_y: {} \n'.format(y.shape))

print('Before OverSampling, counts of label "1": {}'.format(sum(y==1)))
print('Before OverSampling, counts of label "0": {} \n'.format(sum(y==0)))

print('After OverSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_resampled.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_resampled==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_resampled==0)))

In [ ]:
def oversampling(X, y):
    # SMOTE 객체
    sm = SMOTE(random_state = 100)

    # oversampling
    X_rs, y_rs = sm.fit_resample(X, y)

    return X_rs, y_rs

# 4.Stratified KFold 학습 

In [ ]:
#stratified_learn함수 정의

def stratified_learn(classification_models, num_split, random_state = 100):
    #전체 데이터프레임 생성
    df_total = pd.DataFrame({"인덱스": range(0,1095)})

    #각 모델별로 stratified k-fold 학습 및 예측 수행
    for model in classification_models:
        # StratifiedKFold
        skf = StratifiedKFold(n_splits = num_split, shuffle = True, random_state = random_state)
        idx_list = []
        proba_list = []

        for i, (train_index, test_index) in enumerate(skf.split(X, y)):
            X_train, y_train = X.iloc[train_index], y.iloc[train_index]
            X_test, y_test = X.iloc[test_index], y.iloc[test_index]

            #Oversampling작업
            # X_train_rs, y_train_rs = oversampling(X_train, y_train)

            #Scaling작업
            ##############
            
            #XGBClassifier 학습 및 확률값 예측
            # model.fit(X_train_rs, y_train_rs)
            model.fit(X_train, y_train)            
            proba = model.predict_proba(X_test)[:,1]

            #test_index, proba를 리스트에 추가
            idx_list.extend(test_index)
            proba_list.extend(proba)
            
            print(f"\n{model.__class__.__name__}의{i+1}번째 fold 예측 완료")
            print("인덱스 개수", len(idx_list), "예측값 수", len(proba_list))

        df_model = pd.DataFrame({"인덱스": idx_list,
                                 f"{model.__class__.__name__}의 P(y=1)": proba_list})
        # print(df_model)
        df_total = pd.merge(df_total, df_model, how = 'inner', on = '인덱스')
            
        print(f"\n{model.__class__.__name__}의 데이터프레임 생성 완료")

    df_total = df_total.sort_values("인덱스").set_index("인덱스")
    df_total["평균 P(y=1)의 확률"] = df_total.mean(axis = 1)
    print("\n최종 데이터프레임 생성 완료")

    return df_total   

In [ ]:
# pd.DataFrame({"피처" : xgboost.feature_names_in_, "중요도" : xgboost.feature_importances_}).sort_values('중요도', ascending = False)

In [ ]:
# 분류 모델 리스트
classification_models = [randomforest, xgboost, lgbm, catboost]
df_total = stratified_learn(classification_models, num_split=3, random_state = 100)

In [ ]:
df_total

In [ ]:
pd.options.display.float_format = '{:.7f}'.format
df_total.sort_values("XGBClassifier의 P(y=1)", ascending = False)

In [ ]:
df_total.corr()

In [ ]:
import seaborn as sns
import plotly.express as px

plt.figure(figsize=(3,1))
px.box(df_total["평균 P(y=1)의 확률"])

In [ ]:
df_total[df_total["평균 P(y=1)의 확률"]>0.5].shape

In [ ]:
df["전기차충전기유무"].value_counts()

In [ ]:
threshold = 0.5
prediction = np.where(df_total["평균 P(y=1)의 확률"] > threshold, 1, 0)
print("f1값:", f1_score(y, prediction))
plt.figure(figsize = (5,5))
cm_matrix = confusion_matrix(y, prediction, labels= [0,1])
sns.heatmap(cm_matrix, annot=True, fmt = 'd')
plt.show()

In [ ]:
feature_imp = pd.DataFrame({"feature":xgboost.feature_names_in_,
                            "randomforest중요도" :randomforest.feature_importances_,
                            "xgboost중요도" :xgboost.feature_importances_,
                            "lgbm중요도" :lgbm.feature_importances_,
                            "catboost중요도" :catboost.feature_importances_})

In [ ]:
feature_imp

In [ ]:
from sklearn.metrics import precision_score, recall_score

In [ ]:
thres_list = np.arange(0, 1, 0.1)
precision_list = []
recall_list = []

for thres in thres_list:
    prediction = np.where(df_total["평균 P(y=1)의 확률"] > thres, 1, 0)
    precision = precision_score(y, prediction)
    recall = recall_score(y, prediction)

    precision_list.append(precision)
    recall_list.append(recall)

plt.figure(figsize=(10,5))
sns.lineplot(x = thres_list, y = precision_list, label = "precision")
sns.lineplot(x = thres_list, y = recall_list, label = "recall")
plt.legend()
plt.show()

In [ ]:
prediction = np.where(df_total["평균 P(y=1)의 확률"] > 0.5, 1, 0)
df_total["prediction_0.22"] = prediction.reshape(-1,1)

In [ ]:
df_total["actual"] = y

In [ ]:
df_total

,XGBClassifier의 P(y=1),LGBMClassifier의 P(y=1),CatBoostClassifier의 P(y=1),평균 P(y=1)의 확률,prediction_0.22,actual,confusion
인덱스,,,,,,,
0,0.5563816,0.5882781,0.5863778,0.5770125,1,0,TN
1,0.0048319,0.0000015,0.1244611,0.0430982,0,0,TN
2,0.0033828,0.0000004,0.6939414,0.2324415,1,0,TN
3,0.1551217,0.0470225,0.6016362,0.2679268,1,0,TN
4,0.0007603,0.0000000,0.0338786,0.0115463,0,0,TN
...,...,...,...,...,...,...,...
1090,0.0768837,0.0006426,0.4441777,0.1739013,0,0,TN
1091,0.2593538,0.1574427,0.8161100,0.4109689,1,0,TN
1092,0.0148800,0.0000026,0.5562942,0.1903923,0,0,TN


In [ ]:
def confusion(x):
    if (x["actual"] == 1) and (x["prediction_0.22"] == 1):
        return "TP"
    elif (x["actual"] == 0) and (x["prediction_0.22"] == 0):
        return "TN"
    elif (x["actual"] == 1) and (x["prediction_0.22"] == 0):
        return "FN"
    else:
        return "FP"

In [ ]:
confusion_lst = []
for i in df_total.values:
    if (i[4] == 1) and (i[5] == 1):
        confusion_lst.append("TP")
    elif (i[4] == 0) and (i[5] == 0):
        confusion_lst.append("TN")
    elif (i[4] == 1) and (i[5] == 0):
        confusion_lst.append("FP")
    elif (i[4] == 0) and (i[5] == 1):
        confusion_lst.append("FN")

In [ ]:
df_total["confusion"] = confusion_lst

In [ ]:
df_total[df_total["prediction_0.22"] == 1]

,XGBClassifier의 P(y=1),LGBMClassifier의 P(y=1),CatBoostClassifier의 P(y=1),평균 P(y=1)의 확률,prediction_0.22,actual,confusion
인덱스,,,,,,,
0,0.5563816,0.5882781,0.5863778,0.5770125,1,0,FP
2,0.0033828,0.0000004,0.6939414,0.2324415,1,0,FP
3,0.1551217,0.0470225,0.6016362,0.2679268,1,0,FP
8,0.8670504,0.2001493,0.8582926,0.6418308,1,0,FP
9,0.0538675,0.9374023,0.4625987,0.4846228,1,1,TP
...,...,...,...,...,...,...,...
1081,0.4700207,0.0119907,0.4683415,0.3167843,1,0,FP
1086,0.7914025,0.9985152,0.6314799,0.8071325,1,0,FP
1089,0.7637722,0.9818164,0.6096551,0.7850813,1,1,TP


In [ ]:
df_total[(df_total["confusion"] == "FP") | (df_total["confusion"] == "TP")].sort_values(by = "평균 P(y=1)의 확률", ascending = False)

,XGBClassifier의 P(y=1),LGBMClassifier의 P(y=1),CatBoostClassifier의 P(y=1),평균 P(y=1)의 확률,prediction_0.22,actual,confusion
인덱스,,,,,,,
1065,0.9838566,0.9999582,0.9124633,0.9654260,1,0,FP
16,0.9958016,0.9999092,0.8816274,0.9591127,1,1,TP
623,0.9750931,0.9996133,0.8933925,0.9560330,1,0,FP
113,0.9806226,0.9987995,0.8871468,0.9555230,1,1,TP
1020,0.9407344,0.9997773,0.8326077,0.9243732,1,1,TP
...,...,...,...,...,...,...,...
1023,0.1568913,0.0018596,0.5084033,0.2223848,1,0,FP
235,0.1546004,0.0314049,0.4810130,0.2223394,1,1,TP
105,0.1305389,0.0798230,0.4562910,0.2222177,1,0,FP


In [ ]:
df_total.to_csv("1차필터링.csv", index = False)

In [ ]:
import spacy

sentence = "The first method failed because it caused the wires to melt, but the second method succeeded in bending the wires without causing the same issue."
nlp = spacy.load("en_core_web_sm")
doc = nlp(sentence)
s_v_count = 0
token_list = [token.pos_ for token in doc]
if (token_list.count("CCONJ") > 0) and (token_list.count("SCONJ") > 0):
    print("Compound Complex")
elif (token_list.count("SCONJ") > 0): 
    print("Complex")







# return "Compound Complex"

    # else ()
# print(token_list)

Compound Complex
